### **Handling missing values**

---


#### **1. Complete Case Analysis (CCA)**


---


#### **2. Imputation**


##### **1. Univariate**


**i. Numerical Data**


**ii. Categorical Data**


---


##### **2. Multivariate**


**i. KNN Imputer**


**ii. Iterative Imputer**


---
